<a href="https://colab.research.google.com/github/Belal-AI/FSDAwaesf/blob/main/Detr_Facebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import nn
from PIL import Image
from torchvision.models import resnet50
import torch
import requests
import torchvision.transforms as T
import torchvision

In [2]:
class DETR_demo(nn.Module):
    def __init__(self, num_classes, n_heads=8, embedd_dim=256) -> None:
        super().__init__()

        self.backbone = resnet50()      # the backbone ifn the detre is the resnet50 and its output 2048 before the fc
        del self.backbone.fc

        self.conv = nn.Conv2d(2048, embedd_dim, 1)  ## to reduce the number of channels
        self.object_quer=nn.Parameter(torch.rand(300,1,embedd_dim),requires_grad=True) ## position encoding for the encoder input

        self.flatten=nn.Flatten(start_dim=2,end_dim=3) ##flat the input for transformer to mekr just 3 dims(len_Seq,batch,embeddings)

        self.transformer=nn.Transformer(embedd_dim,n_heads,6,6)

        self.bbx_layer=nn.Linear(embedd_dim,4)
        self.classifier=nn.Linear(embedd_dim,num_classes+1)

    def forward(self, x):
        """ the process path"""
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x=self.conv(x)

        x=self.flatten(x)
        print(x.permute(2,0,1).shape)
        print(self.object_quer.shape)

        trans=self.transformer(x.permute(2,0,1),self.object_quer)

        bbx=self.bbx_layer(trans)

        cls=self.classifier(trans)


        #print(trans.shape)
        return bbx,cls


In [3]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
im = Image.open(requests.get(url, stream=True).raw)
print(im.size)
transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
im=transform(im).unsqueeze(0)
im.shape

(640, 480)


torch.Size([1, 3, 480, 640])

In [4]:
model=DETR_demo(10)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [5]:
optmizer=torch.optim.AdamW(model.parameters())
criterion = nn.CrossEntropyLoss()

In [6]:
optmizer.zero_grad()
bbx,cls=model.forward(im)
loss_bbx=torchvision.ops.distance_box_iou(bbx[0],torch.tensor([100, 50, 200, 150]).unsqueeze(0))
loss_cls=criterion(cls[0],torch.tensor([0]))+loss_bbx
loss_cls.backward()
optmizer.step()


torch.Size([300, 1, 256])
torch.Size([300, 1, 256])
